In [1]:
from active_critic.utils.gym_utils import *
from gym.spaces.box import Box

/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [104]:
class MultiEnvWrapper(gym.Wrapper):
    def __init__(self, list_envs) -> None:
        super().__init__(list_envs[0])
        self.list_envs = list_envs
        self.current_env = 1
        self._make_observation_space()
        print('init__________________')


    def reset(self):
        print(self.current_env)
        self.current_env = (self.current_env + 1) % len(self.list_envs)
        print('reset')
        print(self.current_env)
        self._make_observation_space()
        obs =  super().reset()
        obs = np.append(obs, self.current_env)
        return obs

    def step(self, action):

        obs, rew, done, info = super().step(action)
        obs = np.append(obs, self.current_env)

        return obs, rew, done, info

    def _make_observation_space(self):
        low = self.list_envs[self.current_env].observation_space.low
        high = self.list_envs[self.current_env].observation_space.high
        dtype = self.list_envs[self.current_env].observation_space.dtype
        new_low = np.append(low, 0)
        new_high = np.append(high, len(self.list_envs))
        new_obs_space = Box(new_low, new_high, dtype=dtype)
        self.observation_space = new_obs_space
        self.action_space = self.list_envs[self.current_env].action_space

def make_env_list(env_ids):
    policy_dict = make_policy_dict()
    list_envs = []
    list_experts = []
    for i in range(len(env_ids)):
        exp, env_id = policy_dict[env_ids[i]]
        env = ALL_V2_ENVIRONMENTS_GOAL_OBSERVABLE[env_id]()
        env._freeze_rand_vec = False
        list_envs.append(env)
        list_experts.append(exp)
    return list_envs, list_experts
    
class MultiImitationLearningWrapper:
    def __init__(self, policies, env: GymEnv):
        self.observation_space = env.observation_space
        self.action_space = env.action_space
        self.policies = policies

    def predict(self, obsv, deterministic=None):
        actions = []
        for obs in obsv:
            current_env = int(obs[-1])
            print(f'current_env: {current_env}')
            inpt_obsv = obs[:-1]
            print(inpt_obsv.shape)
            actions.append(self.policies[current_env].get_action(inpt_obsv))
        return actions

In [105]:
def make_multi_vec_env(env_ids, num_cpu, seq_len, sparse):

    def make_env(env_ids, rank, seed=0):
        def _init():
            max_episode_steps = seq_len
            list_envs, list_experts = make_env_list(env_ids)
            multi_env = MultiEnvWrapper(list_envs=list_envs)
            timelimit = TimeLimit(env=multi_env, max_episode_steps=max_episode_steps)
            strict_time = StrictSeqLenWrapper(timelimit, seq_len=seq_len + 1, sparse=sparse)
            riw = RolloutInfoWrapper(strict_time)
            return riw
        return _init
        
    env = SubprocVecEnv([make_env(env_ids, i) for i in range(num_cpu)])

    list_envs, list_experts = make_env_list(env_ids)
    vec_expert = MultiImitationLearningWrapper(
        policies=list_experts, env=env)
    return env, vec_expert

In [106]:
multi_vec, multi_exp = make_multi_vec_env(env_ids=['reach', 'pickplace', 'reach'], num_cpu=2, seq_len=100, sparse=False)

/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


init__________________
init__________________


In [108]:
obs = multi_vec.reset()


2
reset
0
2
reset
0


In [ ]:
mu

In [95]:
obs = multi_vec.reset()
done = False
while not done:
    actions = multi_exp.predict(obs)
    obs, rew, dones, info = multi_vec.step(actions)
    done = dones[0]
    print(rew)
#print(rew)

1
reset
0
1
reset
0
current_env: 0
current_env: 0
[1.44714296 1.34519083]
current_env: 0
current_env: 0
[1.47191442 1.36587317]
current_env: 0
current_env: 0
[1.52252455 1.40611551]
current_env: 0
current_env: 0
[1.59990789 1.46527749]
current_env: 0
current_env: 0
[1.70434467 1.54220544]
current_env: 0
current_env: 0
[1.83648177 1.63606676]
current_env: 0
current_env: 0
[1.99752404 1.74661723]
current_env: 0
current_env: 0
[2.18881762 1.87423754]
current_env: 0
current_env: 0
[2.40394701 2.02011946]
current_env: 0
current_env: 0
[2.63900891 2.18332527]
current_env: 0
current_env: 0
[2.90183745 2.3604594 ]
current_env: 0
current_env: 0
[3.19604311 2.55318824]
current_env: 0
current_env: 0
[3.52268337 2.76544116]
current_env: 0
current_env: 0
[3.88098341 3.00124488]
current_env: 0
current_env: 0
[4.26844261 3.26312836]
current_env: 0
current_env: 0
[4.68070552 3.55149096]
current_env: 0
current_env: 0
[5.11154774 3.86480007]
current_env: 0
current_env: 0
[5.55317151 4.19986069]
current_

In [ ]:
obs = multi_vec.reset()

In [ ]:
obs

In [ ]:
multi_vec.reset()

In [ ]:
b = np.concatenate((a, np.array([100]), a[10:]))

In [ ]:
b